# Structured outputs
利用できるモデルとAPIバージョンが限られています。
https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/structured-outputs

In [1]:
!python -m pip install openai --upgrade #1.42.0以上が必要 
!python -m pip install python-dotenv
!python -m pip install pydantic --upgrade #2.8.2以上が必要。JSONのオブジェクトスキーマを定義するためのライブラリ
import json
from openai import AzureOpenAI
from dotenv import load_dotenv
import os
from pydantic import BaseModel

In [12]:
load_dotenv()
deployment = os.environ['DPLOYMENT']
apikey = os.environ['API_KEY']
base = os.environ['BASE']
api_version="2024-08-01-preview" #2024-08-01-preview以上にする必要あり

In [14]:
client = AzureOpenAI(
  azure_endpoint = base, 
  api_key=apikey,  
  api_version=api_version
)

# Structured outputsの実践

In [15]:
##構造として出力してほしいスキーマのモデル
class IngredientsToMakeDish(BaseModel):
    dishName: str
    existingIngredients: list[str]
    needToBuyIngredients: list[str]
#呼び出すメソッドが通常のChatCompletionとは異なるので注意
completion = client.beta.chat.completions.parse( 
    model=deployment,
    messages=[
        {"role": "system", "content": "Extract the dish name and ingredients that already have and need to buy information."},
        {"role": "user", "content": "今日はぶり大根を作ろうと思います。ブリは家にありますが、ネギと大根がないです。そういえばぶり大根には普通入れないタコもあるので入れてみましょう。"},
    ],
    response_format=IngredientsToMakeDish
)
print(completion.choices[0].message)

ParsedChatCompletionMessage[IngredientsToMakeDish](content='{"dishName":"ぶり大根","existingIngredients":["ブリ","タコ"],"needToBuyIngredients":["ネギ","大根"]}', refusal=None, role='assistant', function_call=None, tool_calls=[], parsed=IngredientsToMakeDish(dishName='ぶり大根', existingIngredients=['ブリ', 'タコ'], needToBuyIngredients=['ネギ', '大根']))


In [17]:
ingredientsToMakeDish = completion.choices[0].message.parsed
print(ingredientsToMakeDish)

dishName='ぶり大根' existingIngredients=['ブリ', 'タコ'] needToBuyIngredients=['ネギ', '大根']
